# PyRAMS XArray tutorials

First, import needed packages

In [ ]:
import xarray as xr
import pyrams.xarray
from glob import glob

Get list of data

In [ ]:
flist = sorted(glob('data/*.h5'))
flist

Open dataset with xarray. We'll use `xr.open_mfdataset()` to open multiple files as a single dataset. We'll concat the files along a new dimension called `time`

In [ ]:
ds = xr.open_mfdataset(flist, combine='nested', concat_dim='time')

By default, the spatial dimensions are called `phony_dim_0, phony_dim_1...`. No coordinates exist

In [ ]:
ds

A variable, such as `RCP` (cloud liquid mass) does not have any metadata

In [ ]:
ds.RCP

## Use PyRAMS to add metadata and coordinates to our dataset

First, add named dimensions (x, y, and z) and x/y/z/time coordinates. This method uses the timestamps in each file's name (provided by `flist`) to generate `datetime64` date and time information. `dx` and `dz` set horizontal and vertical grid spacing, respectively. Instead of `dz`, `z=...` can be passed with an explicit list/array of `z` heights (like from ztn in the `*-HEAD.txt` RAMS output). 

_ToDo: add functionality to automatically read `z` heights from the HEAD files_

In [ ]:
ds = ds.rams.fix_dims(flist=flist, dx=62.5, dz=6.25)

Now, our dataset has properly named `x,y,z` dimensions and proper coordinate variables

In [ ]:
ds

And our variable now shows proper dimension names

In [ ]:
ds.RCP

Now we can also add variable metadata (units, long_name) to all our variables. This information comes from a [json file contained within the PyRAMS package](https://github.com/lsterzinger/pyrams/blob/main/pyrams/rams-vars.json)

In [ ]:
ds.rams.apply_variable_metadata()

Now our variable has `long_name` and `units` attributes

In [ ]:
ds.RCP

Plotting with xarray now includes labels, as well as selection dimensions by name (`dim=('x', 'y')` and `x='time'`)

In [ ]:
ds.RCP.plot()

In [ ]:
ds.RCP.mean(dim=('x', 'y')).plot(x='time', add_labels=True)

and more easily select/slice data

In [ ]:
import numpy as np
t = np.datetime64('2019-07-02T00:05:00')

ds.RCP.sel(z=600,time=t).plot()

## Use Metpy to parse units and perform unit-aware calculations

In [ ]:
import metpy.xarray

In [ ]:
cloud_number = ds.CCP.metpy.quantify()
density = ds.DN0.metpy.quantify()

In [ ]:
print(cloud_number.metpy.units)
print(density.metpy.units)

In [ ]:
cloud_n_per_volume = cloud_number * density

In [ ]:
cloud_n_per_volume.metpy.convert_units('1/cm^3').sel(time=t).mean(dim=('x','y')).plot(y='z')